In [2]:
#!pip install langchain==0.0.208 deeplake openai==0.27.8 tiktoken

In [1]:
from platform import python_version

print(python_version())

3.11.5


In [2]:
from langchain.llms import OpenAI

In [5]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

In [8]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..




Day 1:
Warm up: 10 minutes of light jogging or brisk walking
Main workout:
- 3 sets of 10 lunges
- 3 sets of 10 push-ups
- 3 sets of 10 squats
- 3 sets of 10 bench dips
- 3 sets of 10 step-ups on bench
Cool down: 10 minutes of stretching

Day 2:
Warm up: 10 minutes of jumping jacks
Main workout:
- 20 minute run/jog outdoors
- 3 sets of 10 burpees
- 3 sets of 10 mountain climbers
- 3 sets of 10 bicycle crunches
- 3 sets of 10 jumping lunges
Cool down: 10 minutes of stretching

Day 3:
Warm up: 10 minutes of jump rope
Main workout:
- 3 sets of 10 tricep dips on park bench
- 3 sets of 10 pull-ups on playground equipment
- 3 sets of 10 box jumps
- 3 sets of 10 Russian twists with medicine ball
- 3 sets of 10 jump squats
Cool down: 10 minutes of stretching

Day 4:
Rest day or low


## The Chains

In [2]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))

AuthenticationError: Incorrect API key provided: sk-E6ryT***************************************xqgw. You can find your API key at https://platform.openai.com/account/api-keys.

# The Memory

In [11]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Well, I am an artificial intelligence created by a team of programmers and engineers. My purpose is to assist and interact with humans in various tasks and conversations. I am constantly learning and improving through algorithms and data analysis. I am currently housed in a server room with other

# Deep Lake VectorStore

In [ ]:
import os
os.environ["ACTIVELOOP_TOKEN"] = "<YOUR-ACTIVELOOP-TOKEN>"

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# instantiate the LLM and embeddings models
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "<YOUR-ACTIVELOOP-ORG-ID>" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

In [ ]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions"
        ),
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
response = agent.run("when was napoleone born?")
print(response)

In [ ]:
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

texts = [
    "lafya Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

db.add_documents(docs)

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions",
    ),
]

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
response = agent.run("when was Michael Jordan born?")

# Agents in LangChain

In [ ]:
from langchain.agents import initialize_agent, Tool, load_tools, AgentType
from langchain.llms import OpenAI
from langchain.utilities import GooglePlacesAPIWrapper

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

In [ ]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

In [ ]:
tools = [
    Tool(
        name="Google Search",
        func=search.run,
        description="Useful for when uou need to search google to answer question about current events",
    )
]

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=6)

In [ ]:
response = agent("What's the latest new about the Mars Rover?")
print(response['output'])

# Tools in LangChain

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])